# 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2022-06-24

In [ ]:
# 2


# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")

# 사용자 코딩
# print(X_test.isnull().sum())
# print(X_train.isnull().sum())
# print(y_train.isnull().sum())
X_test = X_test.drop(["cust_id"], axis = 1)
X_train = X_train.drop(["cust_id"], axis = 1)
# print(X_test.head())

y = y_train[["gender"]]
# print(y.head())
X_test_cat = X_test[["주구매상품", "주구매지점"]]
X_test_not_cat = X_test[["총구매액", "최대구매액", "환불금액", "내점일수", "내점당구매건수", "주말방문비율", "구매주기"]]
X_test_not_cat = X_test_not_cat.fillna(0)

X_train_cat = X_train[["주구매상품", "주구매지점"]]
X_train_not_cat = X_train[["총구매액", "최대구매액", "환불금액", "내점일수", "내점당구매건수", "주말방문비율", "구매주기"]]
X_train_not_cat = X_train_not_cat.fillna(0)

X_test_cat = pd.get_dummies(X_test_cat)
X_train_cat = pd.get_dummies(X_train_cat)

X_train_cat, X_test_cat = X_train_cat.align(X_test_cat, join = "inner", axis = 1)

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(X_train_not_cat)
X_scaled_train_not_cat = scaler.transform(X_train_not_cat)
X_scaled_test_not_cat = scaler.transform(X_test_not_cat)

final_train = pd.concat([pd.DataFrame(X_scaled_train_not_cat), X_train_cat], axis = 1)
final_test = pd.concat([pd.DataFrame(X_scaled_test_not_cat), X_test_cat], axis = 1)

from sklearn.model_selection import train_test_split
x_train, x_valid, Y_train, Y_valid = train_test_split(final_train, y, random_state = 42, stratify = y, test_size = 0.2)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 42)
model.fit(x_train, Y_train)
prob_valid = model.predict_proba(x_valid)
print(f"train score : {model.score(x_train, Y_train)}")
print(f"valid score : {model.score(x_valid, Y_valid)}")

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier(random_state = 420)
model2.fit(x_train, Y_train)
prob_valid2 = model.predict_proba(x_valid)
print(f"train score : {model.score(x_train, Y_train)}")
print(f"valid score : {model.score(x_valid, Y_valid)}")

from sklearn.svm import SVC
model3 = SVC(random_state = 4200, probability = True)
model3.fit(x_train, Y_train)
prob_valid3 = model.predict_proba(x_valid)
print(f"train score : {model.score(x_train, Y_train)}")
print(f"valid score : {model.score(x_valid, Y_valid)}")

from sklearn.linear_model import LogisticRegression
model4 = LogisticRegression(random_state = 42000)
model4.fit(x_train, Y_train)
prob_valid4 = model.predict_proba(x_valid)
print(f"train score : {model.score(x_train, Y_train)}")
print(f"valid score : {model.score(x_valid, Y_valid)}")

from sklearn.metrics import roc_auc_score
print(f"roc_auc test score : {roc_auc_score(Y_valid, prob_valid[:, 1])}")
print(f"roc_auc test score : {roc_auc_score(Y_valid, prob_valid2[:, 1])}")
print(f"roc_auc test score : {roc_auc_score(Y_valid, prob_valid3[:, 1])}")
print(f"roc_auc test score : {roc_auc_score(Y_valid, prob_valid4[:, 1])}")

total_prob_valid = (prob_valid[:, 1] + prob_valid2[:, 1] + prob_valid3[:, 1] + prob_valid4[:, 1]) / 4
print(total_prob_valid)

prob_test = model.predict_proba(final_test)
prob_test2 = model2.predict_proba(final_test)
prob_test3 = model3.predict_proba(final_test)
prob_test4 = model4.predict_proba(final_test)

total_prob_test = (prob_test[:, 1] + prob_test2[:, 1] + prob_test3[:, 1] + prob_test4[:, 1]) / 4

X_test = pd.read_csv("data/X_test.csv")
# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
final_ans = pd.DataFrame({'custid': X_test.cust_id, 'gender': total_prob_test})
# print(final_ans)
final_ans.to_csv('2016003790.csv', index=False)

data_hutatabi = pd.read_csv("2016003790.csv")
print(data_hutatabi.head(10))
print("OWari")


